In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv
/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv


In [2]:
!pip install kaggle

In [3]:
import os

os.environ["KAGGLE_USERNAME"] = "ernidhi"
os.environ["KAGGLE_KEY"] = "KGAT_1c8e2ae620ede4cb27a4f19a633c0a60"

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("EcommerceWorkspace") \
    .enableHiveSupport() \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 07:44:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark.sparkContext.setLogLevel("ERROR")


In [6]:
SparkSession.builder.getOrCreate()

In [7]:
spark.sql("""
CREATE SCHEMA IF NOT EXISTS ecommerce
""")

DataFrame[]

In [8]:
spark.sql("SHOW DATABASES").show()


+---------+
|namespace|
+---------+
|  default|
|ecommerce|
+---------+



In [9]:
!mkdir -p data/ecommerce


In [10]:
spark.sql("""
CREATE DATABASE IF NOT EXISTS ecommerce_data
""")

DataFrame[]

In [11]:
!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store


Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
100%|██████████████████████████████████████▉| 4.29G/4.29G [00:04<00:00, 459MB/s]
100%|██████████████████████████████████████| 4.29G/4.29G [00:04<00:00, 1.08GB/s]


In [12]:
!unzip ecommerce-behavior-data-from-multi-category-store.zip -d ecommerce_data

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: ecommerce_data/2019-Nov.csv  
  inflating: ecommerce_data/2019-Oct.csv  


In [13]:
!cd ecommerce_data && ls -lh


total 14G
-rw-r--r-- 1 root root 8.4G Dec  9  2019 2019-Nov.csv
-rw-r--r-- 1 root root 5.3G Dec  9  2019 2019-Oct.csv


In [14]:
!rm -f ecommerce-behavior-data-from-multi-category-store.zip


In [15]:
!ls -lh




total 60K
drwxr-xr-x 3 root root 4.0K Jan 11 07:44 data
-rw-r--r-- 1 root root  686 Jan 11 07:44 derby.log
drwxr-xr-x 2 root root 4.0K Jan 11 07:50 ecommerce_data
drwxr-xr-x 5 root root 4.0K Jan 11 07:44 metastore_db
---------- 1 root root  37K Jan 11 07:51 __notebook__.ipynb
drwxr-xr-x 4 root root 4.0K Jan 11 07:44 spark-warehouse


In [16]:
df_n = spark.read.csv("/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv")

In [17]:
df = spark.read.csv("/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv")

In [18]:
print(f"October 2019 - Total Events: {df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df.printSchema()

October 2019 - Total Events: 42,448,765

SCHEMA:
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



In [19]:
print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
df.show(5, truncate=False)


SAMPLE DATA (First 5 rows):
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+------+---------+------------------------------------+
|_c0                    |_c1       |_c2       |_c3                |_c4                                |_c5     |_c6   |_c7      |_c8                                 |
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code                      |brand   |price |user_id  |user_session                        |
|2019-10-01 00:00:00 UTC|view      |44600062  |2103807459595387724|NULL                               |shiseido|35.79 |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00 UTC|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.20 |554748717|9333dfb

In [20]:
def load_ecommerce_data(month):
    """
    Load ecommerce behavior data for a given month.
    
    Parameters:
        month (str): Month identifier like 'Oct', 'Nov'
    
    Returns:
        Spark DataFrame
    """
    path = f"ecommerce_data/*{month}*.csv"
    
    df = spark.read.csv(
        path,
        header=True,
        inferSchema=True
    )
    
    return df


In [21]:
def load_ecommerce_data(month):
    """
    Load ecommerce behavior data for a given month.
    """
    path = f"/kaggle/input/ecommerce-behavior-data-from-multi-category-store/*-{month}.csv"
    
    df = spark.read.csv(
        path,
        header=True,
        inferSchema=True
    )
    
    return df


In [22]:
# Load your data
events = load_ecommerce_data("Oct")
# Verify it's working
print(f"✅ Ready to go! Loaded {events.count():,} events")
events.show(5)

# Your Day 1 challenges start here...


✅ Ready to go! Loaded 42,448,764 events
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    NULL|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      vi

In [23]:
# Create simple DataFrame
data = [("iPhone", 999), ("Samsung", 799), ("MacBook", 1299)]
df = spark.createDataFrame(data, ["product", "price"])
df.show()

# Filter expensive products
df.filter(df.price < 1000).show()


+-------+-----+
|product|price|
+-------+-----+
| iPhone|  999|
|Samsung|  799|
|MacBook| 1299|
+-------+-----+

+-------+-----+
|product|price|
+-------+-----+
| iPhone|  999|
|Samsung|  799|
+-------+-----+



In [24]:
events.filter(events.event_type == 'view').show(10)

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    NULL|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      view|   1004237|2053013555631882655|electr

In [25]:
events.select('product_id','brand','price').filter(events.price.cast('double')>1000).show(10)

+----------+-------+-------+
|product_id|  brand|  price|
+----------+-------+-------+
|   1004237|  apple|1081.98|
|   1005135|  apple|1747.79|
|   1005135|  apple|1747.79|
|   1005105|  apple|1415.48|
|   1005073|samsung|1207.71|
|   1306083|     hp|1512.78|
|   1480456| pulser|1016.73|
|   1005135|  apple|1747.79|
|   1004237|  apple|1081.98|
|   1005072|samsung|1044.09|
+----------+-------+-------+
only showing top 10 rows



In [26]:
events.select('brand','price').show()

+--------+-------+
|   brand|  price|
+--------+-------+
|shiseido|  35.79|
|    aqua|   33.2|
|    NULL|  543.1|
|  lenovo| 251.74|
|   apple|1081.98|
|  pulser| 908.62|
|   creed| 380.96|
|luminarc|  41.16|
|   baden| 102.71|
|  huawei| 566.01|
|elenberg|  51.46|
| samsung| 900.64|
|   haier| 102.38|
|shiseido|  35.79|
|     brw|  93.18|
|    NULL| 357.79|
|   haier| 193.03|
|   bosch|  58.95|
|      hp| 580.89|
|   apple|1747.79|
+--------+-------+
only showing top 20 rows



In [27]:
events.filter(events.price.cast('double')>1000).count()

2447782